In [1]:
#!pip install azure-cognitiveservices-vision-customvision
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry
import numpy as np

# Create Project

In [2]:


ENDPOINT = "<endpointhere>"

# Replace with a valid key
training_key = "<trainkeyhere>"
publish_iteration_name = "classifyBirdModel"

trainer = CustomVisionTrainingClient(training_key, endpoint=ENDPOINT)

# Create a new project
print ("Creating project...")
project = trainer.create_project("Bird Classification")
print("Project created!")

Creating project...
Project created!


# Create Tag list from file names

In [3]:
#Create Tag List from folders in bird directory
import os
os.chdir('./bird_photos')
tags = [name for name in os.listdir('.') if os.path.isdir(name)]
print(tags)

['American Crow', 'American Goldfinch (Female)', 'American Goldfinch (Male)', 'American Robin (Adult)', 'American Robin (Juvenile)', 'Blue Jay', 'Common Grackle', 'House Sparrow (Female)', 'House Sparrow (Male)', 'House Wren', 'Mourning Dove', 'Northern Cardinal (Adult Male)', 'Northern Cardinal (Female)', 'Red-tailed Hawk (Dark morph)', 'Red-tailed Hawk (Light morph immature)', 'Tufted Titmouse']


# Create tag in Azure

In [4]:
def createTag(tag):
    result = trainer.create_tag(project.id, tag)
    print(f'{tag} create with id: {result}')
    return result.id


# Create Image List to Upload

In [10]:
def createImageList(tag, tag_id):
    #os.chdir('../')
    #os.chdir('./bird_photos/') 
    #set directory to current tag
    base_image_url = f"./{tag}/"
    os.chdir(base_image_url) 
    photo_name_list = os.listdir()
    
    print("Adding images...")

    image_list = []

    for file_name in photo_name_list:
        with open(file_name, "rb") as image_contents:
            image_list.append(ImageFileCreateEntry(name=file_name, contents=image_contents.read(), tag_ids=[tag_id]))
    return image_list

# Upload List method

In [6]:
def uploadImageList(image_list):  
    upload_result = trainer.create_images_from_files(project.id, images=image_list)
    if not upload_result.is_batch_successful:
        print("Image batch upload failed.")
        for image in upload_result.images:
            print("Image status: ", image.status)
        exit(-1)

# Loop thru tags and call each method to upload

In [11]:
#get images names from directory
for tag in tags:
    tag_id = createTag(tag)
    print(f"tag creation done with tag id {tag_id}")
    image_list = createImageList(tag, tag_id)
    print("image_list created with length " + str(len(image_list)))
    
    #break list into lists of 25 and upload in batches
    for i in range(0, len(image_list), 25):
        broken = image_list[i:i + 25]
        print(f'Upload started for batch {i} total items {len(broken)} for tag {tag.name}...')
        uploadImageList(broken)
        print(f"Batch {i%25} Image upload completed. Total uploaded {len(broken)} for tag {tag}")

CustomVisionErrorException: Name not unique: American Crow, Project: 20e5bd7c-9677-4ae3-b8e2-4177a7f2ca72

In [14]:
#os.getcwd()

In [13]:
os.chdir('./bird_photos/') 

In [ ]:
os.chdir('../') 